In [83]:
from six.moves import urllib
import pandas as pd
from bs4 import BeautifulSoup

In [153]:
url = "https://arxiv.org/list/astro-ph/recent"

In [154]:
years = ['10','11','12','13','14','15','16','17','18','19','20','21']
months = ['01','02','03','04','05','06','07','08','09','10','11','12']

In [155]:
year = years[0]
month = months[0]

In [163]:
#url of arxiv page with first 25 papers in the month
url25 = "https://arxiv.org/list?archive=astro-ph&year=" + year + "&month=" + month +"&submit=Go" 
url25

'https://arxiv.org/list?archive=astro-ph&year=10&month=01&submit=Go'

In [157]:
#find number of papers that month
page25 = urlopen(url25)

In [160]:
page25
soup25 = BeautifulSoup(html, "html.parser")

In [187]:
print soup25

<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>Astrophysics  authors/titles recent submissions</title>
<link href="/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="//static.arxiv.org/static/browse/0.3.2.8/css/arXiv.css?v=20220215" media="screen" rel="stylesheet" type="text/css"/>
<link href="/bibex/bibex.css?20181009" media="screen" rel="stylesheet" type="text/css"/>
<link href="https://static.arxiv.org/static/browse/0.3.8/css/browse_search.css" media="screen" rel="stylesheet" type="text/css">
<link href="http://arxiv.org/rss/astro-ph" rel="alternate" title="Astrophysics " type="application/rss+xml"/>
<script src="/js/mathjaxToggle.min.js" type="text/javascript"></script>
</link></head>
<body class="with-cu-identity">
<div id="cu-identity">
<div id="cu-logo">
<a href="https://www.cornell

In [199]:
n = soup25.find_all("small")
total_papers = (n[0].text.split("total of ")[1]).split(" entries")[0]
total_papers = int(total_papers)
int(total_papers)

323

In [201]:
urlall = "https://arxiv.org/list/astro-ph/" + year + month + "?show=" + total_papers


#e.g. in February 2022 there were 1205 papers so the url is 
#https://arxiv.org/list/astro-ph/2202?show=1205

urlall

u'https://arxiv.org/list/astro-ph/1001?show=323'

In [24]:
page = urlopen(urlall)
soup = BeautifulSoup(html, "html.parser")

In [151]:
#find all <a> tags
a = soup.find_all("a", href=True)

In [152]:
authors=[]
for a in soup.find_all('a', href=True):
    if 'author' in a['href']:
        authors.append(a.string)
        

In [121]:
#take first name only 
firstNames = []
for ai in authors:
    firstNames.append(ai.split(' ')[0])

#***need to handle initials***

In [147]:
#read in lists of male and female names
femaleNames = pd.read_table("femaleNames.txt",skiprows=1)
maleNames = pd.read_table("maleNames.txt",skiprows=1)

#convert to list
femaleNames = femaleNames['Name'].tolist()
maleNames = maleNames['Name'].tolist()

In [148]:
n_female=0
n_male=0

for name in firstNames:
    if name in femaleNames:
        n_female += 1
    if name in maleNames:
        n_male += 1